In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
dune_length = 15
dune_width = 10
collapse_threshold = 4


In [4]:
#I'm thinking that since my sand is wrapping around the edges, I can drop sand in the zero point without loss of generality. I.e. I could just visualize it at the centre at the end.
#THis is a first play around with just looping through the function in array form from the top left corner.
dune_map = np.zeros((dune_length, dune_width))

def central_sand_drop_first(dune_map, total_sand):
    center_x = 0
    center_y = 0
    for i in range(total_sand):
        dune_map[center_x, center_y] += 1
        for (i, j), value in np.ndenumerate(dune_map):
            # I need to work out how to propogate through the entire map until no more collapses are needed.Does it matter the order in which I do it?
            # because i've set the drop point to the top left, this loop will correctly start there. 
            if value >= collapse_threshold:
                sand_pile_collapse_modular(dune_map, i, j)
    return dune_map

def sand_pile_collapse_modular(dune_map, x_loc, y_loc):
    dune_map[x_loc, y_loc] -= 4
    dune_map[(x_loc - 1)%dune_length, y_loc] += 1
    dune_map[(x_loc + 1)%dune_length, y_loc] += 1
    dune_map[x_loc, (y_loc - 1)%dune_width] += 1
    dune_map[x_loc, (y_loc + 1)%dune_width] += 1
    return dune_map

central_sand_drop_first(dune_map, 16)


array([[4., 0., 1., 0., 0., 0., 0., 0., 1., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0., 2.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0., 2.]])

The above cell seems to vaguely work. However, there are clearly several existing issues which will need to be addressed one way or another
- The edges of the cell are not edges, this means that we will have edge effects once the sand propogates beyond dune_length/2
- It is not addressed how to approach the timeline of the sand stacks. I.e. the if a sand pile collapses onto 4 adjacent sandpiles and causes them to collapse- they in turn make it 4 so it should once again collapse on to them! One sneaky work around would be to set the collapse threshold as 5, since they can only gain four from neighbours, would this resolve the problem? - I am not sure actually because let's say you were on 3 and 4 neighbours collapsed to make you 7 then you would collapse to 3, but in principle they could collapse back to you if they had all been raised to 7? I am not sure if this is true, but it would be good to resolve regardless.
-I will need to change the array shape for visualisation so that it does actually show up in the middle.   

Upon thinking further, I wonder if the most natural way to do it is by examining only the four adjacent squares after a collapse. Since none of them are adjacent they cannot interact directly so it should be okay to go to any one of them in order. This seems like it could be done recursively, but I don't want to end up following a whole chain of collapses before returning to the other 3 adjacent to the first one, so this will need to be considered.


In [5]:
# dune_map = np.zeros((dune_length, dune_width))
def central_sand_drop(dune_map, total_sand):
    center_x = dune_length // 2
    center_y = dune_width // 2
    for i in range(total_sand):
        dune_map[center_x, center_y] += 1
        sand_propagator(center_x, center_y, dune_map)
    return dune_map

def sand_propagator(x_loc, y_loc, dune_map):
    if dune_map[x_loc, y_loc] >= collapse_threshold:
        sand_pile_collapse_modular(dune_map, x_loc, y_loc)
        # Now check adjacent squares
        sand_propagator((x_loc - 1)%dune_length, y_loc, dune_map)
        sand_propagator((x_loc + 1)%dune_length, y_loc, dune_map)
        sand_propagator(x_loc, (y_loc - 1)%dune_width, dune_map)
        sand_propagator(x_loc, (y_loc + 1)%dune_width, dune_map)


central_sand_drop(dune_map, 30)

array([[4., 0., 1., 0., 0., 0., 0., 0., 1., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0., 2.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 2., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 3., 0., 1., 0., 0.],
       [0., 0., 0., 2., 3., 2., 3., 2., 0., 0.],
       [0., 0., 0., 1., 0., 3., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 2., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0., 2.]])

This recursive method works well except for the fact that it will follow along chains of collapses rather than considering all four neighbours before moving on to another cell. I wonder if I can use four for loops rather than recursion? Or could I some how add a counter to direct where the next considered point is? What if every time there is a collapse I add coordinates to an array that needs to be looped through in order? 

In [6]:
# dune_map = np.zeros((dune_length, dune_width))
def central_sand_drop_loop(dune_map, total_sand):
    center_x = dune_length // 2
    center_y = dune_width // 2
    for i in range(total_sand):
        dune_map[center_x, center_y] += 1
        sand_propagator_loop(dune_map)
    return dune_map

def sand_propagator_loop(dune_map):
    unstable = True
    while unstable:
        unstable = False
        for (i, j), value in np.ndenumerate(dune_map):
            if value >= collapse_threshold:
                sand_pile_collapse_modular(dune_map, i, j)
                unstable = True
    return dune_map

central_sand_drop_loop(dune_map, 10)

array([[0., 1., 1., 0., 0., 0., 0., 0., 1., 1.],
       [1., 2., 0., 0., 0., 0., 0., 0., 0., 2.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 3., 1., 0., 0., 0.],
       [0., 0., 0., 1., 2., 3., 2., 1., 0., 0.],
       [0., 0., 0., 3., 3., 0., 3., 3., 0., 0.],
       [0., 0., 0., 1., 2., 3., 2., 1., 0., 0.],
       [0., 0., 0., 0., 1., 3., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 2., 0., 0., 0., 0., 0., 0., 0., 2.]])

Okay, this method has some merit (mostly suggested by AI). The mechanics of it is still not perfect because it just looks through i, j in order, i.e if we actually break down the mechanics it is not physical (we could end up chasing a collaps along horizontally to the right and then not returning to the left hand collapse until it has looped through all elements in the array to return back). 

In [7]:
dune_map = np.zeros((dune_length, dune_width))
def central_sand_drop_array(dune_map, total_sand):
    center_x = dune_length // 2
    center_y = dune_width // 2
    for i in range(total_sand):
        dune_map[center_x, center_y] += 1
        sand_propagator_array(dune_map, center_x, center_y)
    return dune_map

def sand_propagator_array(dune_map, x_loc, y_loc):
    search_coords = np.array([[x_loc, y_loc]])
    while len(search_coords) > 0:
        current_coord = search_coords[0]
        search_coords = np.delete(search_coords, 0, axis=0)
        i, j = current_coord
        if dune_map[i, j] >= collapse_threshold:
            sand_pile_collapse_modular(dune_map, i, j)
            # Now add adjacent squares to search coords
            adjacent_coords = np.array([
                [(i - 1)%dune_length, j],
                [(i + 1)%dune_length, j],
                [i, (j - 1)%dune_width],
                [i, (j + 1)%dune_width]
            ])
            search_coords = np.vstack((search_coords, adjacent_coords))
    return dune_map

central_sand_drop_array(dune_map, 51)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 3., 1., 3., 0., 0., 0.],
       [0., 0., 0., 3., 2., 2., 2., 3., 0., 0.],
       [0., 0., 1., 1., 2., 3., 2., 1., 1., 0.],
       [0., 0., 0., 3., 2., 2., 2., 3., 0., 0.],
       [0., 0., 0., 0., 3., 1., 3., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

I think this has the most real world like mechanics - i.e it examines all local collapses before propagating out to further collapses. It will still take a little thinking about how to render this 'frame by frame'. I.e if one collapse causes four more collapses, those four should be simultaneous. 

So, how can I make it do updates at the correct time? I wonder if I can give them a tag which updates at the correct time? Surely there is a way to build it into the function to automatically update at the right time? 
Okay, maybe I have sorted the idea of tagging them. Let's try:

In [10]:
dune_map = np.zeros((dune_length, dune_width))
def central_sand_drop_array_tagged(dune_map, total_sand):
    center_x = dune_length // 2
    center_y = dune_width // 2
    for i in range(total_sand):
        dune_map[center_x, center_y] += 1
        sand_propagator_array_tagged(dune_map, center_x, center_y)
        print(dune_map)
    return dune_map

def sand_propagator_array_tagged(dune_map, x_loc, y_loc):
    search_coords = np.array([[x_loc, y_loc, 0]])
    while len(search_coords) > 0:
        current_coord = search_coords[0]
        search_coords = np.delete(search_coords, 0, axis=0)
        i, j, tag = current_coord
        if dune_map[i, j] >= collapse_threshold:
            sand_pile_collapse_modular(dune_map, i, j)
            # Now add adjacent squares to search coords
            adjacent_coords = np.array([
                [(i - 1)%dune_length, j, tag + 1],
                [(i + 1)%dune_length, j, tag + 1],
                [i, (j - 1)%dune_width, tag +1],
                [i, (j + 1)%dune_width, tag+1]
            ])
            search_coords = np.vstack((search_coords, adjacent_coords))
    return dune_map

central_sand_drop_array_tagged(dune_map, 51)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 3., 1., 3., 0., 0., 0.],
       [0., 0., 0., 3., 2., 2., 2., 3., 0., 0.],
       [0., 0., 1., 1., 2., 3., 2., 1., 1., 0.],
       [0., 0., 0., 3., 2., 2., 2., 3., 0., 0.],
       [0., 0., 0., 0., 3., 1., 3., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])